In [5]:
import os, json, cv2
from pathlib import Path
from ultralytics import YOLO

def run_detection(input_folder, model_path, output_folder, logs_folder):
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    Path(logs_folder).mkdir(parents=True, exist_ok=True)

    model = YOLO(model_path)
    all_results = []

    for img_file in Path(input_folder).glob("*"):
        if img_file.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue

        img = cv2.imread(str(img_file))
        if img is None:
            print(f"Warning: Could not read {img_file}")
            continue

        results = model(str(img_file))
        dets = []

        for box in results[0].boxes:
            cls_index = int(box.cls[0])
            label = model.names[cls_index]

            # Simulate gloved/bare detection: only use "person" class for demo
            if label != "person":
                continue

            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Fake logic for demo: even x1 -> gloved, odd x1 -> bare
            hand_label = "gloved_hand" if x1 % 2 == 0 else "bare_hand"

            dets.append({"label": hand_label, "confidence": conf, "bbox": [x1, y1, x2, y2]})
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, f"{hand_label} {conf:.2f}", (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        cv2.imwrite(str(Path(output_folder)/img_file.name), img)

        log = {"filename": img_file.name, "detections": dets}
        with open(Path(logs_folder)/(img_file.stem+".json"), "w") as f:
            json.dump(log, f, indent=2)
        all_results.append(log)

    with open(Path(logs_folder)/"all_detections.json", "w") as f:
        json.dump({"images": all_results}, f, indent=2)

    print("Done! Check", output_folder, "and", logs_folder)


if __name__ == "__main__":
    input_folder = input("Enter path to image folder (e.g., ./sample_images): ")
    model_path = input("Enter YOLO model path (.pt, e.g., yolov8n.pt): ")

    run_detection(
        input_folder=input_folder,
        model_path=model_path,
        output_folder="./output",
        logs_folder="./logs"
    )


Enter path to image folder (e.g., ./sample_images):  ./sample_images
Enter YOLO model path (.pt, e.g., yolov8n.pt):  yolov8n.pt



image 1/1 E:\coding\Data Science\opencv\glove\sample_images\images.jpg: 352x640 (no detections), 213.9ms
Speed: 6.5ms preprocess, 213.9ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 E:\coding\Data Science\opencv\glove\sample_images\img1.jpg: 448x640 1 person, 204.3ms
Speed: 5.0ms preprocess, 204.3ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 E:\coding\Data Science\opencv\glove\sample_images\img2.jpg: 512x640 1 person, 221.5ms
Speed: 4.4ms preprocess, 221.5ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 E:\coding\Data Science\opencv\glove\sample_images\img3.jpg: 480x640 1 person, 235.6ms
Speed: 3.5ms preprocess, 235.6ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 E:\coding\Data Science\opencv\glove\sample_images\img4.jpg: 640x544 1 person, 183.2ms
Speed: 5.4ms preprocess, 183.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 E:\co